## Visualization 5: Relationship between NBA Player of the Week and Team Location

Note: This notebook requires `maps` and `mapdata` which could be installed by running `install.packages(c("maps", "mapdata"))` in R

Note: This notebook requires `ggnewscale` which could be installed by running `devtools::install_github("eliocamp/ggnewscale")` in R

Note: This notebook requires `us` which could be installed by running `pip install us` in Python

In [1]:
%load_ext rpy2.ipython
%R library(ggplot2)
%R library(maps)
%R library(mapdata)
%R library(ggnewscale)

import pandas as pd
import numpy as np
import us
from utils import pd2r

Load Data and calculate POTW count for each team

In [2]:
location = pd2r(pd.read_csv('../data/scraped/NBA_Location.csv'))
nba_Abb = pd.read_csv('../data/scraped/NBA_Teams_Abb.csv')
potw = pd.read_csv('../data/raw/NBA_Player_of_the_Week.csv')
nba_colors = pd.read_csv('../data/scraped/NBA_Teams_Color.csv')
potw = potw.loc[potw['Season short'] > 1990]

In [3]:
# Add Abbreviation for the team
location_w_abb = (pd.merge(location, nba_Abb, left_on = 'Team', right_on = 'Franchise', how = 'left').drop(columns = 'Franchise'))
location_w_abb = location_w_abb.rename(columns = {'Abbreviation/Acronym':'Abb'})
location_w_abb.head()

,Division,Team,"City, State",Arena,Capacity,Coordinates,Founded,Joined,latitude,longitude,Abb
0,Atlantic,Boston Celtics,"Boston, Massachusetts",TD Garden,18624,﻿42.366303°N 71.062228°W,1946,1946,42.366303,-71.062228,BOS
1,Atlantic,Brooklyn Nets,"New York City, New York",Barclays Center,17732,﻿40.68265°N 73.974689°W,1967*,1976,41.000000,-75.000000,BKN
2,Atlantic,New York Knicks,"New York City, New York",Madison Square Garden,19812,﻿40.750556°N 73.993611°W,1946,1946,40.750556,-73.993611,NYK
3,Atlantic,Philadelphia 76ers,"Philadelphia, Pennsylvania",Wells Fargo Center,21600,﻿39.901111°N 75.171944°W,1946*,1949,39.901111,-75.171944,PHI
4,Atlantic,Toronto Raptors,"Toronto, Ontario",Scotiabank Arena,19800,﻿43.643333°N 79.379167°W,1995,1995,43.643333,-79.379167,TOR


In [4]:
# Count how many times each NBA team got POTW in history
potw_count = potw.groupby('Team').count()['Player'].reset_index()
potw_count.loc[potw_count.Team == 'Philadelphia Sixers', 'Team'] = 'Philadelphia 76ers'
location_new = location_w_abb[['Team','latitude','longitude','Abb']]

In [5]:
# map POTW count to locatin
loc_potw_ct = pd.merge(location_new,potw_count, on = 'Team', how = 'left')
loc_potw_ct.head()

,Team,latitude,longitude,Abb,Player
0,Boston Celtics,42.366303,-71.062228,BOS,34
1,Brooklyn Nets,41.000000,-75.000000,BKN,6
2,New York Knicks,40.750556,-73.993611,NYK,30
3,Philadelphia 76ers,39.901111,-75.171944,PHI,32
4,Toronto Raptors,43.643333,-79.379167,TOR,34


Map Team color to the data

In [6]:
nba_df = pd.merge(loc_potw_ct, nba_colors, left_on = 'Team', right_on = 'Franchise', how = 'left')
nba_colors = nba_df.set_index('Team')
nba_ggplot_fill = nba_colors['Color1']
nba_ggplot_color = nba_colors['Color2']
nba_df.head()

,Team,latitude,longitude,Abb,Player,Franchise,Tm,Color1,Color2
0,Boston Celtics,42.366303,-71.062228,BOS,34,Boston Celtics,BOS,#007A33,#BA9653
1,Brooklyn Nets,41.000000,-75.000000,BKN,6,Brooklyn Nets,BKN,#000000,#000000
2,New York Knicks,40.750556,-73.993611,NYK,30,New York Knicks,NYK,#006BB6,#F58426
3,Philadelphia 76ers,39.901111,-75.171944,PHI,32,Philadelphia 76ers,PHI,#006BB6,#ED174C
4,Toronto Raptors,43.643333,-79.379167,TOR,34,Toronto Raptors,TOR,#CE1141,#000000


Map color to the region for ggplot

In [7]:
# import states of USA
states = np.array([state.name.lower() for state in us.states.STATES])
states = pd.DataFrame(states).rename(columns = {0:'State'})
states.head()

,State
0,alabama
1,alaska
2,arizona
3,arkansas
4,california


In [8]:
# Extract the state for each team
region = location_w_abb.copy()[['Division', 'City, State']]
region['State'] = region['City, State'].str.split(', ').str[1].str.lower()
region.loc[region.State == 'd.c.', 'State'] = 'district of columbia'
region = region.drop(columns = ['City, State']).drop_duplicates()

In [9]:
# Merge two table
TM_location = pd.merge(states,region, on = 'State', how = 'left').fillna('No_NBA')
# Map color to each state
color_dic = {}
color_lst = ["gray90","darkseagreen2","lightblue","lightpink","thistle2","wheat","tan1"]
i = 0
for loc in TM_location.Division.unique():
    color_dic[loc] = color_lst[i]
    i +=1
TM_location['color'] = TM_location['Division'].map(color_dic)
location_color = TM_location.rename(columns = {'State': 'region'}).set_index('region')['color']
location_color.head()

region
alabama              gray90
alaska               gray90
arizona       darkseagreen2
arkansas             gray90
california    darkseagreen2
Name: color, dtype: object

Visualization in R

In [10]:
%R -i nba_df
%R -i nba_ggplot_color
%R -i nba_ggplot_fill
%R -i location_color

In [11]:
%%R -w 10 -h 5 --units in -r 300
usa <- map_data('state')
canada <- map_data('worldHires', 'Canada')

ggplot() +
geom_polygon(data = usa, aes(x = long, y = lat, group = group, fill = region),
             color = 'gray', size = 0.7) +
scale_fill_manual(values = location_color) +
guides(fill = F) +
new_scale('fill') +
geom_polygon(data = canada, aes(x = long, y = lat, group = group),
             fill = 'grey90', color = 'gray', size = 0.5) +
geom_point(data = nba_df, aes(x = longitude, y = latitude, size = Player, color = Team, fill = Team),
           shape = 21, alpha = 0.7) +
scale_size(name = 'Number of times\nreceiving\nPlayer of the Week', range = c(1, 10)) +
new_scale('size') +
geom_text(data = nba_df, aes(x = longitude, y = latitude, label = Abb, size = Player), 
          family = 'Avenir Next', fontface = 'bold', color = 'white') +
scale_size(range = c(0, 3)) +
# geom_label(data = nba_df, aes(x = longitude, y = latitude, label = Abb, color = Team, fill = Team), alpha = 0.8, size = 2,position = 'dodge') +
scale_color_manual(values = nba_ggplot_color) +
scale_fill_manual(values = nba_ggplot_fill) +
coord_fixed(xlim = c(-125, -70), ylim = c(25, 50), ratio = 1.25) +
theme_minimal() +
theme(text = element_text(family = 'Avenir Next'),
      axis.text = element_blank(),
      axis.line = element_blank(),
      axis.ticks = element_blank(),
      panel.border = element_blank(),
      panel.grid = element_blank(),
      axis.title = element_blank(),
      legend.title = element_text(face = 'bold', size = 10),
      legend.position = 'None',
      legend.background = element_rect(size = 0, fill = 'snow2')) +
guides(fill = F, color = F, size = F, size = F, shape = F)

ggsave('../vis/vis5.png', units = 'in', width = 10, height = 5)